In [1]:
import psycopg2
import pandas as pd
import sys
sys.path.append("..")
import credentials
from sqlalchemy import create_engine
engine = create_engine(credentials.postgres)
import numpy as np
from datetime import datetime
import string
from tqdm import tnrange, tqdm_notebook
# This makes plots render inline

In [2]:
% matplotlib inline

In [3]:
# Basic Connection and Query
connection = engine.connect()

In [4]:
key_codes = pd.read_csv('key_codes.csv').set_index('code').key

In [5]:
def get_sequence(connection=connection, filter_alpha=False):
    people = {}
    # Calculate Dwells
    result = connection.execute("SELECT * FROM strokes WHERE user_id='1d63b44d-a7cb-4ee6-b228-b0ff5b7d086f' ORDER BY key_time")
    dwells = []
    for row in result:
        # ('1d63b44d-a7cb-4ee6-b228-b0ff5b7d086f', 1509862331813, '3675', '40964', 'd')
        row = (row[0], row[1], int(row[2]), int(row[3]), row[4])
        # Assure Users are in Dictionaries
        if row[0] not in people:
            people[row[0]] = {}
            
        if row[2] not in people[row[0]] and row[4] == 'd':
            people[row[0]][row[2]] = row[1]
        elif row[2] in people[row[0]] and row[4] == 'd':
            people[row[0]][row[2]] = row[1]
        elif row[2] in people[row[0]] and row[4] == 'u':
            dwell = row[1] - people[row[0]][row[2]]
            try:
                if not filter_alpha or (filter_alpha and key_codes[row[2]] in string.ascii_uppercase):
                    dwells.append({
                        'person': row[0],
                        'key_code': row[2],
                        'key': key_codes[row[2]],
                        'down': people[row[0]][row[2]],
                        'up': row[1],
                        'dwell': dwell
                    })
            except Exception as e:
                pass
                
            del(people[row[0]][row[2]])
    return pd.DataFrame(dwells).sort_values('down')


In [8]:
df = get_sequence()

In [45]:
message = []
for k in df.key:
    if k in string.ascii_uppercase or k.isdigit():
        message.append(k)
    elif k == 'ENTER':
        message.append('\n')
    elif k == 'PERIOD':
        message.append('.')
    elif k == 'BACKSPACE':
        message.pop()
    elif k == 'SPACE':
        message.append(' ')
    elif k == 'COMMA':
        message.append(',')
    elif k == 'SEMICOLON':
        message.append(';')
    elif k == 'TAB':
        message.append('\t')
    elif k == 'MINUS':
        message.append('-')
    elif k == 'SLASH':
        message.append('/')

In [46]:
text = ''.join(message)